# Защита персональных данных клиентов

Необходимо защитить данные клиентов страховой компании «Хоть потоп». Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.

Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from pyspark.ml import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
pth1 = '/datasets/insurance.csv'
data = pd.read_csv(pth1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [3]:
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')

## Умножение матриц

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? 

**Ответ:** Не изменится

**Обоснование:** Посмотрим, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Пусть $P$ - обратимая матрица

Тогда признаки $Z = XP$

Тогда подставим новые признаки в формклы предсказания :

$$
w1 = (Z^T Z)^{-1} Z^T y
$$

$$
a1 = Zw1 = Z (Z^T Z)^{-1} Z^T y
$$

Учтём, что $(Z^T Z)^{-1} = Z^{-1} (Z^T)^{-1}$

Тогда: 

$$
a1 = Z Z^{-1} (Z^T)^{-1} Z^T y
$$

Заменим $Z = XP$

Тогда: 

$$
a1 = XP (XP)^{-1} ((XP)^T)^{-1} (XP)^T y
$$





Учтём, что $(XP)^{-1} ((XP)^T)^{-1} = ((XP)^T XP)^{-1}$




Тогда: 

$$
a1 = XP ((XP)^T XP)^{-1} (XP)^T y = XP (P^T (X^T X) P)^{-1} (XP)^T y = XP ((X^T X) P)^{-1} (P^T)^{-1} (XP)^T y 
$$

$$
a1 =  XP P^{-1} (X^T X)^{-1} (P^T)^{-1} (XP)^T y = X (X^T X)^{-1} (P^T)^{-1} (XP)^T y
$$



$$
a1 = X (X^T X)^{-1} (P^T)^{-1} P^T X^T y  = X (X^T X)^{-1} X^T y
$$

Вспомним, что $w = (X^T X)^{-1} X^T y$

Тогда 

$$
a1 = Xw = a
$$

**Таким образом предсказание 
$$
a = a1
$$**

## Алгоритм преобразования

**Необходимо:**

**1) составить обратимую матрицу $P$**

Обратная матрица существует только для квадратных матриц, где детерминант не равен 0. Чтобы преобразованная матрица $Z$ имела ту же размерность, что $X$, матрица $P$ должна иметь размерность n на n, где n - количество признаков в $X$

**2) получить преобразованную матрицу $Z = XP$**

In [4]:
#функция для преобразования матрицы
def z_features(features):
    n = features.shape[1]
    np.random.seed(12345)
    p_matrix = np.random.randint(1, 10, (n,n))
    det = np.linalg.det(p_matrix)
    while det == 0:
        np.random.seed(12345)
        p_matrix = np.random.randint(1, 10, (n,n))
        det = np.linalg.det(p_matrix)
    features_z = features @ p_matrix
    return features_z, p_matrix


## Проверка алгоритма

In [5]:
features = data.drop('Страховые выплаты',axis=1)
target = data['Страховые выплаты']



In [6]:
#применим функцию
features_2, p_matrix = z_features(features)

#### Посмотрим данные до преобразования

In [7]:
features.head(5)

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


#### Посмотрим данные после преобразования и матрицу преобразования

In [8]:
features_2.head(5)

,0,1,2,3
0,99452,396931,347287,49899
1,76279,304140,266095,38329
2,42174,168087,147058,21203
3,83532,333667,291948,41861
4,52371,208890,182758,26301


In [9]:
p_matrix

array([[3, 6, 2, 5],
       [6, 3, 2, 7],
       [2, 8, 7, 1],
       [3, 2, 3, 7]])

### Проверка алгоритма на оригинальных данных

In [10]:
#разделим данные
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345)

In [11]:
model = LinearRegression()

model.fit(features_train, target_train)

R2_orig = r2_score(target_test, model.predict(features_test))

print("R2 =", R2_orig)

R2 = 0.4117683938910953


### Проверка алгоритма на преобразованных данных

In [12]:
#разделим данные
features_train, features_test, target_train, target_test = train_test_split(
    features_2, target, test_size=0.2, random_state=12345)

In [13]:
model = LinearRegression()

model.fit(features_train, target_train)

R2_z = r2_score(target_test, model.predict(features_test))

print("R2 =", R2_z)

R2 = 0.41176839389106634


## Выводы

   В данной работе был подобран метод шифрования данных с помощью умножения на необратимую матрицу. Метод был проверен на модели линейной регрессии. Данная работа показывает, что предсказания не меняются, но такая обработка позволяет сохранить конфиденциальность данных пользователей. 

   Теоретическая выкладка представлена в разделе 2 данной работы, сама функция для поиска обратимой подходящей матрицы также представлена в работе